- [ ] Write down all the libraries that we need to install
- [ ] Add Try catch for exceptions when moving to .py file

## Important Notes
### Packages to install :
- pip install sounddevice
- pip install numpy
- Install ffmpeg (needed for intsalling local version of whisper)
    - On Ubuntu or Debian
      sudo apt update && sudo apt install ffmpeg
    - On Arch Linux
      sudo pacman -S ffmpeg
    - On MacOs
      brew install ffmpeg
    - On Windows
      winget install ffmpeg
- pip install -U openai-whisper
- pip install anthropic
- pip install dotenv
- pip install python-dotenv
- pip install openai
- pip install pyaudio
- pip install keyboard
- Installing pytorch
  - Install Cuda Toolkit (https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html) -> Only needed for GPU acceleration
  - Install Cudnn (https://developer.nvidia.com/cudnn) -> Only needed for GPU acceleration
  - pInstall pytorch with or without cuda (https://pytorch.org/get-started/locally/)
- pip install PyMuPDF -> To read the pdf files like CV's
### Instructions :
- There needs to be a .env in this notebook's working directory, which contains the api keys.

In [1]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import whisper
from AnthropicWrapper import ClaudeChatCV
from dotenv import load_dotenv, find_dotenv
import os
from openai import OpenAI
import pyaudio
import threading
import time
import keyboard

load_dotenv(find_dotenv())

True

In [2]:
# --- Settings ---
FS = 44100               # Sampling frequency
THRESHOLD = 50          # Volume threshold for silence (adjust this)
SILENCE_DURATION = 1.5   # Seconds of silence before stopping (adjust this)
CHUNK_SIZE = 1024        # Process audio in chunks for efficiency
SPEAKING_SPEED = 1.1     # Speed of speaking

In [3]:
# --- Globals ---
pause_loop = False

In [4]:
# Initialising Claude and ConversationChain
chat_model_name = "claude-3-5-sonnet-20240620"

system_prompt = "You are a friendly enthusiastic chatbot. You are here to help me with whatever tasks I need or to have a nice conversation. Be sure to keep your answers short and concise. Do not delve into super deep details or lists unless asked for."

# Give the path to a CV in your disk
pdf_path = "D:\Hidden Desktop\OneDrive\Cross Device\Jobs Applications\Graduating\CV.pdf"
chat_model = ClaudeChatCV(chat_model_name, system_prompt, pdf_path)

In [5]:
# Initialising whisper
model = whisper.load_model("medium", device="cuda")

In [6]:
# Initialising text2speech
tts_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def stream_tts(input_string):
    def _stream_tts():
        p = pyaudio.PyAudio()
        stream = p.open(format=8,
                        channels=1,
                        rate=round(24_005 * SPEAKING_SPEED),
                        output=True)
        with tts_client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice="nova",
            input=input_string,
            response_format="pcm"
        ) as response:
            for chunk in response.iter_bytes(1024):
                stream.write(chunk)
                
        #print("FINISHED!!!!!!!!!!!!!!!!!!!!")
        thread_done.set()

    thread_done = threading.Event()

    thread = threading.Thread(target=_stream_tts)
    thread.start()
    thread_done.wait()

In [7]:
# --- Functions ---
def is_silent(data):
    rms = np.sqrt(np.mean(data**2))
    print("RMS: ", rms)
    return rms < THRESHOLD

In [8]:
# Initialising speech2text without silence detection
def record_speech():
    print("Recording... Speak now!")
    audio_data = np.array([], dtype=np.int16)  # Initialize empty array

    with sd.InputStream(samplerate=FS, channels=1, dtype='int16') as stream:
        while True:
            chunk, overflowed = stream.read(CHUNK_SIZE)
            if overflowed:
                print("Warning: Input overflowed!")
            audio_data = np.append(audio_data, chunk)

            if pause_loop:
                break
    
    wav.write("g97613g9f0g8.wav", FS, audio_data)

    return "g97613g9f0g8.wav"


In [9]:
def keyboard_listener():
    global pause_loop
    
    def on_key_press(event):
        global pause_loop
        #print("Key pressed: {}" .format(event.name))
        if event.name == "+":
            pause_loop = False
            print("Set to continue on next loop")
        elif event.name == "-":
            pause_loop = True
            print("Set to pause on next loop")
    
    keyboard.on_press(on_key_press)
    keyboard.wait('esc')

listener_thread = threading.Thread(target=keyboard_listener)
listener_thread.start()

In [10]:
while True:
    if not pause_loop:
        # --- Record Speech ---
        time.sleep(0.1)
        print("Recording speech...")
        wav_file = record_speech()

        # --- Speech to Text ---
        print("Converting speech to text...")
        text = model.transcribe(wav_file, language="en")
        print("You said: ", text.get("text"))

        # --- Chatbot ---
        print("Chatting...")
        response = chat_model.chat_with_history_doc(text.get("text"))

        print("Chatbot: ", response)

        # --- Text to Speech ---
        print("Converting text to speech...")
        stream_tts(response)

    else:
        time.sleep(0.1)

Recording speech...
Recording... Speak now!
Set to pause on next loop
Converting speech to text...
You said:   Hi, how's it going?
Chatting...


Exception in thread Thread-11 (_stream_tts):
Traceback (most recent call last):
  File "c:\Users\niran\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\niran\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\niran\AppData\Local\Temp\ipykernel_340\3141505319.py", line 7, in _stream_tts
  File "c:\Users\niran\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyaudio\__init__.py", line 639, in open
    stream = PyAudio.Stream(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\niran\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyaudio\__init__.py", line 441, in __init__
    self._stream = pa.open(**arguments)
                   ^^^^^^^^^^^^^^^^^^^^
TypeError: 'float' object cannot be interpreted as an integer


Chatbot:  Hello! It's going well, thank you for asking. How are you doing today? Is there anything I can help you with or would you like to chat about something in particular?
Converting text to speech...
